In [1]:
import os
%pwd

'c:\\Users\\abdul\\Text Summarizer\\Text-Summarizer\\research'

In [2]:
os.chdir('../')
%pwd

'c:\\Users\\abdul\\Text Summarizer\\Text-Summarizer'

In [3]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen= True)
class ModelTrainerConfig:
    root_dir: Path
    data_path: Path
    model_ckpt: Path
    num_train_epochs: int
    warmup_steps: int
    per_device_train_batch_size: int
    weight_decay: float
    logging_steps: int
    evaluation_strategy: str
    eval_steps: int
    save_steps: float
    gradient_accumulation_steps: int

In [4]:
from TextSummarizer.constants import *
from TextSummarizer.utils.common import read_yaml, create_directories

In [5]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])
        
    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.TrainingArguments
        
        create_directories([config.root_dir])
        
        model_trainer_config = ModelTrainerConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            model_ckpt = config.model_ckpt,
            num_train_epochs=params.num_train_epochs,
            warmup_steps=params.warmup_steps,
            per_device_train_batch_size=params.per_device_train_batch_size,
            weight_decay=params.weight_decay,
            logging_steps=params.logging_steps,
            evaluation_strategy=params.evaluation_strategy,
            eval_steps=params.eval_steps,
            save_steps=params.save_steps,
            gradient_accumulation_steps=params.gradient_accumulation_steps
        )
        return model_trainer_config
        

In [6]:
from transformers import TrainingArguments, Trainer
from transformers import DataCollatorForSeq2Seq
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from datasets import load_dataset, load_from_disk
import torch

c:\Users\abdul\miniconda3\envs\text_sum\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[2024-11-06 21:23:55,915 : INFO : config: PyTorch version 2.5.1 available.]


In [ ]:
class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config


    
    def train(self):
        device = "cuda" if torch.cuda.is_available() else "cpu"
        tokenizer = AutoTokenizer.from_pretrained(self.config.model_ckpt)
        model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(self.config.model_ckpt).to(device)
        seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer, model=model_pegasus)
        
        #loading data 
        dataset_samsum_pt = load_from_disk(self.config.data_path)

        # trainer_args = TrainingArguments(
        #     output_dir=self.config.root_dir, num_train_epochs=self.config.num_train_epochs, warmup_steps=self.config.warmup_steps,
        #     per_device_train_batch_size=self.config.per_device_train_batch_size, per_device_eval_batch_size=self.config.per_device_train_batch_size,
        #     weight_decay=self.config.weight_decay, logging_steps=self.config.logging_steps,
        #     evaluation_strategy=self.config.evaluation_strategy, eval_steps=self.config.eval_steps, save_steps=1e6,
        #     gradient_accumulation_steps=self.config.gradient_accumulation_steps
        # ) 


        trainer_args = TrainingArguments(
            output_dir=self.config.root_dir, num_train_epochs=1, warmup_steps=500,
            per_device_train_batch_size=1, per_device_eval_batch_size=1,
            weight_decay=0.01, logging_steps=10,
            evaluation_strategy='steps', eval_steps=500, save_steps=1e6,
            gradient_accumulation_steps=16
        ) 

        trainer = Trainer(model=model_pegasus, args=trainer_args,
                  tokenizer=tokenizer, data_collator=seq2seq_data_collator,
                  train_dataset=dataset_samsum_pt["train"], 
                  eval_dataset=dataset_samsum_pt["validation"])
        
        trainer.train()

        ## Save model
        model_pegasus.save_pretrained(os.path.join(self.config.root_dir,"pegasus-model-finetuned-on-samsum-data"))
        ## Save tokenizer
        tokenizer.save_pretrained(os.path.join(self.config.root_dir,"tokenizer"))

In [8]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer_config = ModelTrainer(config=model_trainer_config)
    model_trainer_config.train()
except Exception as e:
    raise e


[2024-11-06 21:24:03,633 : INFO : common: yaml file: config\config.yaml loaded successfully]
[2024-11-06 21:24:03,638 : INFO : common: yaml file: params.yaml loaded successfully]
[2024-11-06 21:24:03,639 : INFO : common: Directory created at: artifacts]
[2024-11-06 21:24:03,640 : INFO : common: Directory created at: artifacts/model_trainer]


Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
c:\Users\abdul\miniconda3\envs\text_sum\lib\site-packages\transformers\training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
C:\Users\abdul\AppData\Local\Temp\ipykernel_6244\3262716738.py:33: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(model=model_pegasus, args=trainer_args,
 18%|█▊        | 9/51 [26:34<1:44:39, 149.50s/it]

KeyboardInterrupt: 

[2024-11-06 13:28:46,396 : INFO : common: yaml file: config\config.yaml loaded successfully]
[2024-11-06 13:28:46,398 : INFO : common: yaml file: params.yaml loaded successfully]
[2024-11-06 13:28:46,400 : INFO : common: Directory created at: artifacts]
[2024-11-06 13:28:46,403 : INFO : common: Directory created at: artifacts/data_ingestion]
[2024-11-06 13:29:00,810 : INFO : 4035589776: artifacts/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 23627009
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "2e7409b328d118a1d37018be788babf8bf9640386387da766ac100e96efa3b93"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 1DCD:262D07:AA02F8:B175DE:672B2784
Accept-Ranges: bytes
Date: Wed, 06 Nov 2024 08:28:42 GMT
Via: 1.1 varnish
X-Served-By: cache-mrs1050098-M

EOFError: 

[2024-11-06 13:28:46,396 : INFO : common: yaml file: config\config.yaml loaded successfully]
[2024-11-06 13:28:46,398 : INFO : common: yaml file: params.yaml loaded successfully]
[2024-11-06 13:28:46,400 : INFO : common: Directory created at: artifacts]
[2024-11-06 13:28:46,403 : INFO : common: Directory created at: artifacts/data_ingestion]
[2024-11-06 13:29:00,810 : INFO : 4035589776: artifacts/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 23627009
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "2e7409b328d118a1d37018be788babf8bf9640386387da766ac100e96efa3b93"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 1DCD:262D07:AA02F8:B175DE:672B2784
Accept-Ranges: bytes
Date: Wed, 06 Nov 2024 08:28:42 GMT
Via: 1.1 varnish
X-Served-By: cache-mrs1050098-M

EOFError: 

[2024-11-06 13:28:46,396 : INFO : common: yaml file: config\config.yaml loaded successfully]
[2024-11-06 13:28:46,398 : INFO : common: yaml file: params.yaml loaded successfully]
[2024-11-06 13:28:46,400 : INFO : common: Directory created at: artifacts]
[2024-11-06 13:28:46,403 : INFO : common: Directory created at: artifacts/data_ingestion]
[2024-11-06 13:29:00,810 : INFO : 4035589776: artifacts/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 23627009
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "2e7409b328d118a1d37018be788babf8bf9640386387da766ac100e96efa3b93"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 1DCD:262D07:AA02F8:B175DE:672B2784
Accept-Ranges: bytes
Date: Wed, 06 Nov 2024 08:28:42 GMT
Via: 1.1 varnish
X-Served-By: cache-mrs1050098-M

EOFError: 

In [17]:
import transformers
import accelerate

print("Transformers version:", transformers.__version__)
print("Accelerate version:", accelerate.__version__)

# !pip install --upgrade accelerate
# !pip uninstall -y transformers accelerate
# !pip install transformers accelerate

Transformers version: 4.46.1
Accelerate version: 0.26.0
